In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy.stats as st
import os
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import *
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox
from datetime import datetime, timedelta

pd.set_option('display.float_format','{:.3f}'.format)
%matplotlib inline 

In [80]:
# dataframe données DJU (chaque region)

df_1 = pd.read_excel('1.xlsx',index_col=None)
df_2 = pd.read_excel('2.xlsx',index_col=None)
df_3 = pd.read_excel('3.xlsx',index_col=None)
df_4 = pd.read_excel('4.xlsx',index_col=None)
df_5 = pd.read_excel('5.xlsx',index_col=None)
df_6 = pd.read_excel('6.xlsx',index_col=None)
df_7 = pd.read_excel('7.xlsx',index_col=None)
df_8 = pd.read_excel('8.xlsx',index_col=None)


In [81]:
# union tous dataframe

df = pd.concat([df_1, df_2, df_3, df_4, df_5,df_6,df_7,df_8])

In [82]:
# set index

df = df.set_index('Unnamed: 0')

In [83]:
# set name index_None

df.index.name = None

In [84]:
# suprimer le colonne Total 

df = df.drop('Total',axis=1)

In [85]:
# renomage des colonnes a cause de accents et des lettres majuscules

df = df.rename(columns={"JAN": "Jan", "FÉV": "Feb", "MAR":"Mar","AVR": "Apr","MAI": "May",
                         "JUN": "Jun","JUI": "Jul","AOÛ": "Aug","SEP": "Sep","OCT": "Oct","NOV": "Nov","DÉC": "Dec"})


df.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2021,387.300,295.300,299.400,268.200,168.200,25.900,0.000,0.000,0.000,0.000,0.000,0.000
2020,318.500,248.700,274.400,139.200,107.500,69.800,48.500,34.600,67.300,166.200,219.600,328.600
2019,389.700,284.600,253.900,219.200,159.700,72.500,27.400,39.200,66.600,132.600,285.800,325.700
2018,295.000,392.400,315.100,177.100,129.700,39.200,21.800,42.700,82.300,160.300,272.300,292.000
2017,445.800,284.200,231.000,236.800,110.700,51.400,31.900,43.300,101.000,124.700,289.500,339.000


In [86]:
df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
2021,387.300,295.300,299.400,268.200,168.200,25.900,0.000,0.000,0.000,0.000,0.000,0.000
2020,318.500,248.700,274.400,139.200,107.500,69.800,48.500,34.600,67.300,166.200,219.600,328.600
2019,389.700,284.600,253.900,219.200,159.700,72.500,27.400,39.200,66.600,132.600,285.800,325.700
2018,295.000,392.400,315.100,177.100,129.700,39.200,21.800,42.700,82.300,160.300,272.300,292.000
2017,445.800,284.200,231.000,236.800,110.700,51.400,31.900,43.300,101.000,124.700,289.500,339.000
...,...,...,...,...,...,...,...,...,...,...,...,...
2013,512.900,464.000,367.400,274.700,251.800,96.100,18.500,36.500,84.100,144.900,377.100,392.700
2012,440.600,548.500,266.400,296.800,143.200,59.800,52.500,24.300,88.400,181.300,301.500,434.700
2011,470.100,362.100,331.100,156.100,101.500,88.000,65.200,32.600,47.700,165.900,241.100,391.000
2010,545.000,445.400,396.400,211.200,206.600,84.100,18.700,39.900,107.100,229.000,371.400,495.700


In [87]:
# reset index
df.reset_index(inplace=True)


In [88]:
df

,index,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2021,387.300,295.300,299.400,268.200,168.200,25.900,0.000,0.000,0.000,0.000,0.000,0.000
1,2020,318.500,248.700,274.400,139.200,107.500,69.800,48.500,34.600,67.300,166.200,219.600,328.600
2,2019,389.700,284.600,253.900,219.200,159.700,72.500,27.400,39.200,66.600,132.600,285.800,325.700
3,2018,295.000,392.400,315.100,177.100,129.700,39.200,21.800,42.700,82.300,160.300,272.300,292.000
4,2017,445.800,284.200,231.000,236.800,110.700,51.400,31.900,43.300,101.000,124.700,289.500,339.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2013,512.900,464.000,367.400,274.700,251.800,96.100,18.500,36.500,84.100,144.900,377.100,392.700
100,2012,440.600,548.500,266.400,296.800,143.200,59.800,52.500,24.300,88.400,181.300,301.500,434.700
101,2011,470.100,362.100,331.100,156.100,101.500,88.000,65.200,32.600,47.700,165.900,241.100,391.000
102,2010,545.000,445.400,396.400,211.200,206.600,84.100,18.700,39.900,107.100,229.000,371.400,495.700


In [89]:
# rename the column to year
df.rename(columns={'index': 'year'}, inplace=True)


In [90]:
df

,year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2021,387.300,295.300,299.400,268.200,168.200,25.900,0.000,0.000,0.000,0.000,0.000,0.000
1,2020,318.500,248.700,274.400,139.200,107.500,69.800,48.500,34.600,67.300,166.200,219.600,328.600
2,2019,389.700,284.600,253.900,219.200,159.700,72.500,27.400,39.200,66.600,132.600,285.800,325.700
3,2018,295.000,392.400,315.100,177.100,129.700,39.200,21.800,42.700,82.300,160.300,272.300,292.000
4,2017,445.800,284.200,231.000,236.800,110.700,51.400,31.900,43.300,101.000,124.700,289.500,339.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2013,512.900,464.000,367.400,274.700,251.800,96.100,18.500,36.500,84.100,144.900,377.100,392.700
100,2012,440.600,548.500,266.400,296.800,143.200,59.800,52.500,24.300,88.400,181.300,301.500,434.700
101,2011,470.100,362.100,331.100,156.100,101.500,88.000,65.200,32.600,47.700,165.900,241.100,391.000
102,2010,545.000,445.400,396.400,211.200,206.600,84.100,18.700,39.900,107.100,229.000,371.400,495.700


In [91]:
# melt df into a long format
df2 = df.melt(id_vars='year', var_name='month')


In [92]:
df2

,year,month,value
0,2021,Jan,387.300
1,2020,Jan,318.500
2,2019,Jan,389.700
3,2018,Jan,295.000
4,2017,Jan,445.800
...,...,...,...
1243,2013,Dec,392.700
1244,2012,Dec,434.700
1245,2011,Dec,391.000
1246,2010,Dec,495.700


In [93]:
# create a datatime column and set it as index
df2['date'] = pd.to_datetime(df2['year'].astype(str) + '-' + df2['month'])
df2.set_index('date', inplace=True)


In [94]:
# renomage le colonne value

df2.rename(columns={'value': 'Chauffage'}, inplace=True)

In [95]:
# suprimer des colonnes year, month
df2 = df2.drop(['year','month'],axis=1)

In [96]:
df2

,Chauffage
date,
2021-01-01,387.300
2020-01-01,318.500
2019-01-01,389.700
2018-01-01,295.000
2017-01-01,445.800
...,...
2013-12-01,392.700
2012-12-01,434.700
2011-12-01,391.000


In [97]:
# reset index
df2 = df2.reset_index()

In [98]:
# chagment de type le colonne date
df2['date'] = df2['date'].astype('datetime64[ns]')

In [106]:
# grouper dataframe

data = df2.groupby('date').mean()

In [107]:
# visulation de dataframe data
data

,Chauffage
date,
2009-01-01,373.312
2009-02-01,293.575
2009-03-01,247.175
2009-04-01,157.025
2009-05-01,86.987
...,...
2021-08-01,0.000
2021-09-01,0.000
2021-10-01,0.000


In [109]:
# export dataframe to csv files pour utiliser dans notre notebook final

data.to_csv('data.csv')